# Тестирование

Для проведения тестов в Go используется команда **go test**, которая работает благодаря множеству принятых соглашений для написания тестовых функций. Этот сравнительно простой механизм эффективен для чистого тестирования и естественным образом расширяется на тесты проверки производительности(benchmark).

## Инструмент go test

Команда go test используется в Go для проведения тестов. В папке пакета файлы, которые оканчиваются на **\_test.go** не включаются в сборку при использовании команды go build. Эти файлы содержат тестировочный код, который будет включен в сборку пакета при вызове команды go test.

Внутри \*\_test.go файлов находятся **три вида** функций:
1. Функции для проведения тестирования. Тестовые функции(test function) начинаются на **Test** и проверяют некоторую часть программы на правильную работу; go test вызывает эти функции и выводит результат: PASS или FAIL.
2. Функции для проверки производительности(benchmark). Бенчмарк функции начинаются на **Benchmark** и проверяют производительность некоторых операций; go test выводит среднее время выполнения операций.
3. Демонстрационные функции(example function). Такие функции начинаются на **Example** и используются для показа возможностей пакета.

## Тестовые функции

Каждый тестовый файл должен импортировать пакет **testing**. Тестовые функции должны иметь следующую сигнатуру:

In [ ]:
func Test<Name>(t *testing.T) {
    // ...
}

Необъязательный суффикс Name должен начинаться с большой буквы. Параметр t предоставляет методы для вывода отчёта о проведении теста и логировании допольнительной информации.

Для примера давайте представим, что у нас есть пакет word, который содержит функцию для проверки является ли строка палиндромом:

In [ ]:
package word

func IsPalindrome(s string) bool {
	for i := range s {
		if s[i] != s[len(s)-i-1] {
			return false
		}
	}
	return true
}

Для тестирования этой функции мы должны добавить в ту же папку, где находится этот файл ещё один файл с именем word_test.go, в которую мы добавим две функции:

In [ ]:
package word

import "testing"

func TestPalindrome(t *testing.T) {
	if !IsPalindrome("detartrated") {
		t.Error(`IsPalindrome("detartrated") = false`)
	}
	if !IsPalindrome("kayak") {
		t.Error(`IsPalindrome("kayak") = false`)
	}
}

func TestNonPalindrome(t *testing.T) {
	if IsPalindrome("palindrome") {
		t.Error(`IsPalindrome("palindrome") = true`)
	}
}

Команда go test без явного указания пакета тестирует пакет в текущей папке. Поэтому для тестирования этого пакета необходимо перейти в его папку и вызвать команду go test. Эти тесты пакет проходит, но тесты с неанглийскими буквами или пунктуацией нет:

In [ ]:
func TestFrenchPalindrome(t *testing.T) {
	if !IsPalindrome("été") {
		t.Error(`IsPalindrome("été") = false`)
	}
}

func TestCanalPalindrome(t *testing.T) {
	input := "A man, a plan, a canal: Panama"
	if !IsPalindrome(input) {
		t.Errorf(`IsPalindrome(%q) = false`, input)
	}
}

Для исправления этих ошибок мы можем переписать IsPalindrome так:

In [ ]:
package word

import "unicode"

func IsPalindrome(s string) bool {
	var letters []rune
	for _, r := range s {
		if unicode.IsLetter(r) {
			letters = append(letters, unicode.ToLower(r))
		}
	}
	for i := range letters {
		if letters[i] != letters[len(letters)-i-1] {
			return false
		}
	}
	return true
}

Также в Go является довольно распространённым написание тестов в виде таблицы входных данных и ожидаемого результата. Для нашего примера можно составить такую таблицу для тестирования:

In [ ]:
func TestIsPalindrome(t *testing.T) {
	var tests = []struct {
		input string
		want  bool
	}{
		{"", true},
		{"a", true},
		{"aa", true},
		{"ab", false},
		{"kayak", true},
		{"detartrated", true},
		{"A man, a plan, a canal: Panama", true},
		{"Evil I did dwell; lewd did I live.", true},
		{"Able was I ere I saw Elba", true},
		{"été", true},
		{"Et se resservir, ivresse reste.", true},
		{"palindrome", false},
		{"desserts", false},
	}
	for _, test := range tests {
		if got := IsPalindrome(test.input); got != test.want {
			t.Errorf("IsPalindrome(%q) = %v", test.input, got)
		}
	}
}

Команда go test -v выводит имя и время выполнения каждого теста, а флаг -run, аргументом которого является регулярное выражение, позволяет выбрать определённые тесты для выполнения.

Методы Error и Errorf не останавливают выполнение тестирования, если какой-либо тест будет провален. Если тестирование нужно остановить, то можно воспользоваться методами Fatal и Fatalf.